In [28]:
import asyncio
import aiobotocore
import json
import datetime
from dateutil.tz import tzutc

#AWS_ACCESS_KEY_ID = "xxx"
#AWS_SECRET_ACCESS_KEY = "xxx"

In [31]:
def load_api_secrets(filename):
    """Convenience to load the API key from a json file."""
    try:
        with open(filename, 'r') as fp:
            api_params = json.load(fp)
    except Exception as e:
        print('Failed to load API secrets key: {}'.format(e))
        api_params = None
    return api_params['region_name'], api_params['aws_access_key_id'], api_params['aws_secret_access_key']

In [32]:
api_key_file = "/Users/robincole/Desktop/aws-creds.json"
region_name, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY = load_api_secrets(api_key_file)
region_name

'us-east-1'

In [43]:
async def go(loop):
    bucket = 'test-37fc824c-9417-46ed-a303-a519c4cfbe26'
    filename = 'dummy.bin'
    key = filename

    session = aiobotocore.get_session(loop=loop)
    async with session.create_client('s3', region_name=region_name,
                                   aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                                   aws_access_key_id=AWS_ACCESS_KEY_ID) as client:
        # upload object to amazon s3
        data = b'\x01'*1024
        resp = await client.put_object(Bucket=bucket,
                                            Key=key,
                                            Body=data)
        print(resp)

        # getting s3 object properties of file we just uploaded
        resp = await client.get_object_acl(Bucket=bucket, Key=key)
        print(resp)

        # get object from s3
        response = await client.get_object(Bucket=bucket, Key=key)
        # this will ensure the connection is correctly re-used/closed
        async with response['Body'] as stream:
            assert await stream.read() == data

        # list s3 objects using paginator
        paginator = client.get_paginator('list_objects')
        async for result in paginator.paginate(Bucket=bucket, Prefix=folder):
            for c in result.get('Contents', []):
                print(c)

        # delete object from s3
        resp = await client.delete_object(Bucket=bucket, Key=key)
        print(resp)

In [44]:
loop = asyncio.get_event_loop()

In [45]:
loop.run_until_complete(go(loop))

RuntimeError: This event loop is already running

{'ResponseMetadata': {'RequestId': '68D282C44DD25661', 'HostId': 'eAJXSNKA/q3AEKf6frMTz9VRUagVfjsp3WAjDf88PrivClirNzlgZJA7n9v7YNjCV6yFFtRj1vc=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'eAJXSNKA/q3AEKf6frMTz9VRUagVfjsp3WAjDf88PrivClirNzlgZJA7n9v7YNjCV6yFFtRj1vc=', 'x-amz-request-id': '68D282C44DD25661', 'date': 'Thu, 11 Apr 2019 06:26:08 GMT', 'etag': '"54ac58cc1e2711a1a3d88bce15bb152d"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"54ac58cc1e2711a1a3d88bce15bb152d"'}
{'ResponseMetadata': {'RequestId': 'BD148333735A0370', 'HostId': '+j2h75WOtAOFLqqByLitpI0cYdQMEl3Mr0ujliYix9ZhKKladAf5Z13/FauLV3gtYAy4RXqeG2g=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '+j2h75WOtAOFLqqByLitpI0cYdQMEl3Mr0ujliYix9ZhKKladAf5Z13/FauLV3gtYAy4RXqeG2g=', 'x-amz-request-id': 'BD148333735A0370', 'date': 'Thu, 11 Apr 2019 06:26:08 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Owner': {'D

Success, but note that in Jupyter we always get `RuntimeError: This event loop is already running`